In [ ]:
#############################################################################################################
##### Notebook Analise de Sobrevivencia
##### Baseado em:
## Natural Language Processing with Python (book)
##
## Dataset: http://www-eio.upc.edu/~pau/cms/rdata/csv/survival/lung.csv
##
##############################################################################################################
## Objetivos:
##   Mostrar as vantagens do metodo Survival Analysis: 
##    - Motivacao para a implementacao de explainability
##    - Teoria de suporte para um novo conjuntos de arquiteturas de aprendizagem profunda
##    - Possivel estrategia de analise
###################################################################################################################

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

path_in = 'C:/Users/dealbuqc/Desktop/ontomqol/Datasets/lung/'

In [ ]:
# Leitura dos dados
data = pd.read_csv(path_in+"lung.csv")
data.head()

In [ ]:
# Estatistica basica sobre os dados

data.describe()

In [ ]:
# Histograma
data["age"].hist()

In [ ]:
# Criacao do modelo a ser utilizado e organizacao dos dados

kmf = KaplanMeierFitter()

# If status = 1 , then dead = 0
# If status = 2 , then dead = 1
data.loc[data.status == 1, 'dead'] = 0
data.loc[data.status == 2, 'dead'] = 1
data.head()

In [ ]:
# Treinamento do modelo
# Primeiro argumento: variavel temporal (tempo de sobrevivencia)
# Segundo argumento: indica se o evento de interesse ocorreu

kmf.fit(durations =  data["time"], event_observed = data["dead"])

# Resultado eh dado em forma de tabela
# Removed = Observed + Censored
# Censored = pessoas que nao morreram mas nao trazem informacao relevante para o experimento
# Observed = pessoas que morrera durante o experimento

kmf.event_table

In [ ]:
# Calculo manual das probabilidades de sobrevivencia

# t=0
event_at_0 = kmf.event_table.iloc[0,:]
surv_for_0 = (event_at_0.at_risk - event_at_0.observed)/event_at_0.at_risk

# t=5
event_at_5 = kmf.event_table.iloc[1,:]
surv_for_5 = (event_at_5.at_risk - event_at_5.observed)/event_at_5.at_risk

# t=11
event_at_11 = kmf.event_table.iloc[2,:]
surv_for_11 = (event_at_11.at_risk - event_at_11.observed)/event_at_11.at_risk

# Calculo real das probabilidades em cada t:
surv_after_5 = surv_for_0 * surv_for_5
surv_after_11 = surv_for_0 * surv_for_5 * surv_for_11
print (surv_after_5)
print (surv_after_11)

In [ ]:
# API ja faz este calculo automaticamente

# momento isolado
print (kmf.predict(11))

In [ ]:
# lista the momentos
print (kmf.predict([0,5,11,12]))

In [ ]:
# todos os momentos
print (kmf.survival_function_)

In [ ]:
# Tambem o numero de dias onde em media 50% dos pacientes morreram.
print (kmf.median_survival_time_)

In [ ]:
# Visualizacao por meio de grafico
kmf.plot()
plt.title("The Kaplan-Meier Estimate")
plt.ylabel("Probabilidade dos pacientes estarem vivos")
plt.show()

In [ ]:
print (kmf.confidence_interval_)

# Probabilidade de morte :
# p(1022) = p(0) +......+p(1022)
print (kmf.cumulative_density_)


In [ ]:
# Visualizacao dos dados acima:
kmf.plot_cumulative_density()

In [ ]:
###############################################################################################

In [ ]:
# Aplicacao do tecnica para diferentes grupos

kmf_m = KaplanMeierFitter() 
kmf_f = KaplanMeierFitter() 

Male = data.query("sex == 1")
Female = data.query("sex == 2")

# Primeiro argumento: serie de tempos de sobrevivencia individuais
# Segundo argumento: serie indicando se o evento de interesse ocorreu

kmf_m.fit(durations =  Male["time"],event_observed = Male["dead"] ,label="Male")
kmf_f.fit(durations =  Female["time"],event_observed = Female["dead"], label="Female")

In [ ]:
kmf_m.event_table

In [ ]:
kmf_f.event_table

In [ ]:
# Probabilidade de sobrevivencia para os grupos no tempo t = 11

print (kmf_m.predict(11))
print (kmf_f.predict(11))

In [ ]:
print (kmf_m.survival_function_)
print (kmf_f.survival_function_)

In [ ]:
# Visualizacao grafica do resultado da funcao de sobrevivencia
kmf_m.plot()
kmf_f.plot()
plt.xlabel("Dias que se passaram")
plt.ylabel("Sobrevivencia")
plt.title("KMF")


In [ ]:
# Densidade acumulativa:
### probabilidade de uma pessoa morrer durante o period indicado

print (kmf_m.cumulative_density_)
print (kmf_f.cumulative_density_)

kmf_m.plot_cumulative_density()
kmf_f.plot_cumulative_density()


In [ ]:
# Log-Rank Test
### Objetivo: verificar se existe alguma diferenca significante entre os grupos comparados
#### Hipotese nula: nao existe diferenca entre os grupos
#### obs. lembrando, quanto menor o p, maior a diferenca entre os grupos.
####      Se (p<0.05), entao rejeitamos a hipotese nula


# Define variables :
T=Male['time']
E=Male['dead']
T1=Female['time']
E1=Female['dead']

from lifelines.statistics import logrank_test

results=logrank_test(T,T1,event_observed_A=E, event_observed_B=E1)
results.print_summary()

In [ ]:
###############################################################################################

In [ ]:
# Cox proportional hazard model

from lifelines import CoxPHFitter

data= data.dropna(subset=['inst', 'time', 'status', 'age', 'sex', 'ph.ecog','ph.karno', 'pat.karno', 'meal.cal', 'wt.loss'])

data = data[[ 'time', 'age', 'sex', 'ph.ecog','ph.karno', 'pat.karno', 'meal.cal', 'wt.loss', 'dead']]

cph = CoxPHFitter()
cph.fit(data,"time",event_col="dead")
cph.print_summary()

In [ ]:
#Lembrando: HR = exp(coef) = 1 indica sem efeito
#           HR = exp(coef) < 1 indica reducao do "perigo"
#           HR = exp(codf) > 1 indica aumento do "perigo"

# Outra forma de visualizacao

cph.plot()


In [ ]:
# Grafico da funcao de sobrevivencia
### cada linha representa um individuo

d_data = data.iloc[0:5,:]
cph.predict_survival_function(d_data).plot()

In [ ]:
# Representacao do tempo medio de sobrevivencia

CTE = kmf.conditional_time_to_event_
plt.plot(CTE)

In [5]:
# Exemplo para relacoes temporais (dados longitudinais)

import pandas as pd
from lifelines import CoxPHFitter

# Example dataset with lagged variable
data = {
    'id': [1, 1, 1, 2, 2, 2, 3, 3],
    'event_time': [10, 20, 30, 15, 25, 35, 10, 20],
    'time_since_last_attack': [0, 10, 10, 0, 10, 10, 0, 10],
    'potential_trigger': ['stress', 'weather', 'stress', 'weather', 'stress', 'stress', 'stress', 'weather']
}
df = pd.DataFrame(data)
df = pd.get_dummies(df, columns=['potential_trigger'])
# Fit a recurrent event analysis model with time-dependent covariates
cph = CoxPHFitter(penalizer=0.0001)

#cph.fit(df, 'event_time', event_col=None, cluster_col='id', duration_col='time_since_last_attack')
cph.fit(df, event_col=None, cluster_col='id', duration_col='event_time')

# Print the summary of the model
cph.print_summary()

model,lifelines.CoxPHFitter
duration col,'event_time'
cluster col,'id'
penalizer,0.0001
l1 ratio,0.0
robust variance,True
baseline estimation,breslow
number of observations,8
number of events observed,8
partial log-likelihood,-6.45
time fit was run,2024-03-06 09:21:26 UTC
